In [ ]:
%%capture
# 딥러닝 모델을 사용하기 위한 transformers 패키지 설치
!pip install transformers

In [ ]:
# numpy, pandas, sklearn 등 필요한 라이브러리 import
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold   #sklearn 에서 라이브러리 호출
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 


import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel

import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':256,
    'SEED':41
}

In [ ]:
train =  pd.read_csv('/content/drive/MyDrive/Dacon/22_문장_분류/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Dacon/22_문장_분류/test.csv')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/roberta-small')

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

In [ ]:
traindata_type = train[['ID','문장','확실성']]

In [ ]:
train.loc[(train["확실성"] == "확실"), "확실성"] = 0
train.loc[(train["확실성"] == "불확실"), "확실성"] = 1
#변경된 target 값 확인
train["확실성"].unique()

array([0, 1], dtype=object)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)
folds=[]

for train_idx, valid_idx in skf.split(traindata_type, traindata_type['확실성']):
    train_idx = np.array(list(set(list(train_idx))))
    valid_idx = np.array(list(set(set(valid_idx))))
    folds.append((train_idx, valid_idx))

In [ ]:
# klue/roberta-large 모델 tokenizer 로 train, valid, test 데이터를 정수로 encoding
def convert_data(data_df,case,mask_token):
    global tokenizer
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # tokenize
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, padding='max_length',truncation=True)
       
        # making input mask
        num_zeros = token.count(mask_token)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # making segment
        segment = [0]*SEQ_LEN
 
        # token, mask, segment
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        if case=='train':
          # label values
          targets.append(data_df[LABEL_COLUMN][i])
 
    # convert to array format    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    if case=='train':
      targets = np.array(targets)

    if case=='train':
       return [tokens, masks, segments], targets
    if case=='test':
       return [tokens, masks, segments]

 
# Load data and convert to bert input format
def load_data(pandas_dataframe,case,mask_token):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    if case=='train':
      data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
      data_x, data_y = convert_data(data_df,'train',mask_token)
      return data_x, data_y
    if case=='test':
      data_x = convert_data(data_df,'test',mask_token)
      return data_x

# Define max_len
SEQ_LEN = 30
DATA_COLUMN = "문장"
LABEL_COLUMN = "확실성"
 
# train
train_x0, train_y0 = load_data(train.iloc[folds[0][0]].reset_index(drop=True),'train',1)
train_x1, train_y1 = load_data(train.iloc[folds[1][0]].reset_index(drop=True),'train',1)
train_x2, train_y2 = load_data(train.iloc[folds[2][0]].reset_index(drop=True),'train',1)
train_x3, train_y3 = load_data(train.iloc[folds[3][0]].reset_index(drop=True),'train',1)
train_x4, train_y4 = load_data(train.iloc[folds[4][0]].reset_index(drop=True),'train',1)

valid_x0, valid_y0 = load_data(train.iloc[folds[0][1]].reset_index(drop=True),'train',1)
valid_x1, valid_y1 = load_data(train.iloc[folds[1][1]].reset_index(drop=True),'train',1)
valid_x2, valid_y2 = load_data(train.iloc[folds[2][1]].reset_index(drop=True),'train',1)
valid_x3, valid_y3 = load_data(train.iloc[folds[3][1]].reset_index(drop=True),'train',1)
valid_x4, valid_y4 = load_data(train.iloc[folds[4][1]].reset_index(drop=True),'train',1)

# test
test_x = load_data(test,'test',1)

  0%|          | 0/13232 [00:00<?, ?it/s]

  0%|          | 0/13233 [00:00<?, ?it/s]

  0%|          | 0/13233 [00:00<?, ?it/s]

  0%|          | 0/13233 [00:00<?, ?it/s]

  0%|          | 0/13233 [00:00<?, ?it/s]

...

  0%|          | 0/3308 [00:00<?, ?it/s]

  0%|          | 0/3308 [00:00<?, ?it/s]

  0%|          | 0/3308 [00:00<?, ?it/s]

  0%|          | 0/3308 [00:00<?, ?it/s]

  0%|          | 0/7090 [00:00<?, ?it/s]

In [ ]:
class Klue_RobertaClassifier(tf.keras.Model):
    def __init__(self, num_class):
        super(Klue_RobertaClassifier, self).__init__()

        self.bert = TFRobertaModel.from_pretrained("klue/roberta-small", from_pt=True)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range,seed=41), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        # outputs value : sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

klue_roberta_model = Klue_RobertaClassifier(num_class=2)

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

In [ ]:
for i in range(5):
  print('########## Fold {} : \n'.format(i))

  klue_roberta_model = Klue_RobertaClassifier(num_class=2)

  # Defining loss function, optimizer and metric
  optimizer = AdamWeightDecay(3e-6,weight_decay_rate=1e-4)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  klue_roberta_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

  # Adding an ealrystop to prevent overfitting
  earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=5)

  checkpoint_path = os.path.join('/content/drive/MyDrive/Dacon/22_문장_분류/확실성/','weight_klue_roberta_back_skf_fold_v3{}.h5'.format(i))
  cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

  # Training
  history = klue_roberta_model.fit(globals()['train_x{}'.format(i)],globals()['train_y{}'.format(i)], epochs=50, batch_size=32,
                             validation_data=(globals()['valid_x{}'.format(i)],globals()['valid_y{}'.format(i)]), callbacks=[earlystop_callback, cp_callback])

  klue_roberta_model.load_weights('/content/drive/MyDrive/Dacon/22_문장_분류/확실성/'+'weight_klue_roberta_back_skf_fold_v3{}.h5'.format(i))

  preds=tf.argmax(klue_roberta_model.predict(globals()['valid_x{}'.format(i)]),axis=1)

  print('Validation set ACC: ',accuracy_score(globals()['valid_y{}'.format(i)],preds))
  print('Validation set Confusion Matrix: \n',confusion_matrix(globals()['valid_y{}'.format(i)],preds))

########## Fold 0 : 



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

Epoch 1/50
414/414 [==============================] - ETA: 0s - loss: 0.2664 - accuracy: 0.9185
Epoch 1: val_accuracy improved from -inf to 0.92747, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v30.h5
414/414 [==============================] - 65s 132ms/step - loss: 0.2664 - accuracy: 0.9185 - val_loss: 0.2173 - val_accuracy: 0.9275
Epoch 2/50
414/414 [==============================] - ETA: 0s - loss: 0.2084 - accuracy: 0.9274
Epoch 2: val_accuracy improved from 0.92747 to 0.93140, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v30.h5
414/414 [==============================] - 57s 137ms/step - loss: 0.2084 - accuracy: 0.9274 - val_loss: 0.2033 - val_accuracy: 0.9314
Epoch 3/50
414/414 [==============================] - ETA: 0s - loss: 0.1897 - accuracy: 0.9336
Epoch 3: val_accuracy did not improve from 0.93140
414/414 [==============================] - 54s 130ms/step - loss: 0.1897

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

Epoch 1/50
414/414 [==============================] - ETA: 0s - loss: 0.2752 - accuracy: 0.9166
Epoch 1: val_accuracy improved from -inf to 0.91989, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v31.h5
414/414 [==============================] - 66s 137ms/step - loss: 0.2752 - accuracy: 0.9166 - val_loss: 0.2332 - val_accuracy: 0.9199
Epoch 2/50
414/414 [==============================] - ETA: 0s - loss: 0.2135 - accuracy: 0.9250
Epoch 2: val_accuracy improved from 0.91989 to 0.93077, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v31.h5
414/414 [==============================] - 57s 137ms/step - loss: 0.2135 - accuracy: 0.9250 - val_loss: 0.2159 - val_accuracy: 0.9308
Epoch 3/50
414/414 [==============================] - ETA: 0s - loss: 0.1904 - accuracy: 0.9312
Epoch 3: val_accuracy improved from 0.93077 to 0.93168, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/we

...

Epoch 1/50
414/414 [==============================] - ETA: 0s - loss: 0.2682 - accuracy: 0.9156
Epoch 1: val_accuracy improved from -inf to 0.91868, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v32.h5
414/414 [==============================] - 66s 136ms/step - loss: 0.2682 - accuracy: 0.9156 - val_loss: 0.2300 - val_accuracy: 0.9187
Epoch 2/50
414/414 [==============================] - ETA: 0s - loss: 0.2106 - accuracy: 0.9268
Epoch 2: val_accuracy improved from 0.91868 to 0.92684, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v32.h5
414/414 [==============================] - 57s 137ms/step - loss: 0.2106 - accuracy: 0.9268 - val_loss: 0.2143 - val_accuracy: 0.9268
Epoch 3/50
414/414 [==============================] - ETA: 0s - loss: 0.1887 - accuracy: 0.9330
Epoch 3: val_accuracy did not improve from 0.92684
414/414 [==============================] - 54s 130ms/step - loss: 0.1887

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

Epoch 1/50
414/414 [==============================] - ETA: 0s - loss: 0.2700 - accuracy: 0.9175
Epoch 1: val_accuracy improved from -inf to 0.92110, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v33.h5
414/414 [==============================] - 66s 136ms/step - loss: 0.2700 - accuracy: 0.9175 - val_loss: 0.2301 - val_accuracy: 0.9211
Epoch 2/50
414/414 [==============================] - ETA: 0s - loss: 0.2121 - accuracy: 0.9264
Epoch 2: val_accuracy improved from 0.92110 to 0.92896, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v33.h5
414/414 [==============================] - 57s 137ms/step - loss: 0.2121 - accuracy: 0.9264 - val_loss: 0.2064 - val_accuracy: 0.9290
Epoch 3/50
414/414 [==============================] - ETA: 0s - loss: 0.1925 - accuracy: 0.9326
Epoch 3: val_accuracy improved from 0.92896 to 0.93138, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/we

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

Epoch 1/50
414/414 [==============================] - ETA: 0s - loss: 0.2694 - accuracy: 0.9189
Epoch 1: val_accuracy improved from -inf to 0.92019, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v34.h5
414/414 [==============================] - 66s 136ms/step - loss: 0.2694 - accuracy: 0.9189 - val_loss: 0.2256 - val_accuracy: 0.9202
Epoch 2/50
414/414 [==============================] - ETA: 0s - loss: 0.2110 - accuracy: 0.9272
Epoch 2: val_accuracy improved from 0.92019 to 0.92201, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/weight_klue_roberta_back_skf_fold_v34.h5
414/414 [==============================] - 58s 139ms/step - loss: 0.2110 - accuracy: 0.9272 - val_loss: 0.2147 - val_accuracy: 0.9220
Epoch 3/50
414/414 [==============================] - ETA: 0s - loss: 0.1900 - accuracy: 0.9330
Epoch 3: val_accuracy improved from 0.92201 to 0.92322, saving model to /content/drive/MyDrive/Dacon/22_문장_분류/확실성/we

In [ ]:
# 모델을 불러와서 test 데이터로 예측 수행
for i in range(0,5):
 klue_roberta_model.load_weights('/content/drive/MyDrive/Dacon/22_문장_분류/확실성/'+'weight_klue_roberta_back_skf_fold_v3{}.h5'.format(i))
 globals()['results_{}'.format(i)] = klue_roberta_model.predict(test_x)

# Save forecasts list toward test data
results_test_list=[results_0,results_1,results_2,results_3,results_4]
# results_test_list=[results_0,results_1,results_2]
np.save('/content/drive/MyDrive/Dacon/22_문장_분류/확실성/'+'results_klue_roberta_back_list_v31.npy',results_test_list)

222/222 [==============================] - 8s 36ms/step


In [ ]:
# 확률 높은 예측값 설정
test_pred=tf.argmax((results_test_list[0]+results_test_list[1]+results_test_list[2]+results_test_list[3]+results_test_list[4])/5,axis=1)
# test_pred=tf.argmax((results_test_list[0]+results_test_list[1]+results_test_list[2])/3,axis=1)

In [ ]:
len(test_pred)

7090

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/22_문장_분류/sample_submission.csv')
submission["확실성"] = test_pred

In [ ]:
submission["확실성"].unique()

array([0, 1])

In [ ]:
# 조정했던 target 값을 원래 평점대로 원상복구
submission.loc[(submission["확실성"] == 1), "확실성"] = "불확실"
submission.loc[(submission["확실성"] == 0), "확실성"] = "확실"
#변경된 target 값 확인
submission["확실성"].unique()

array(['확실', '불확실'], dtype=object)

In [ ]:
submission.to_csv("./submission_klue_확실성_v4.csv", index=False)